In [1]:
import pandas as pd
import numpy as np

In [2]:
perfil = "Jair Bolsonaro"
#perfil = "Lula"
#perfil = "Geral"

In [3]:
if perfil == 'Lula':
  df = pd.read_excel('outputs/Google/5. Posts+tags-normalizado-lula-manual.xlsx')
elif perfil == "Jair Bolsonaro":
  df = pd.read_excel("outputs/Google/5. Posts+tags-normalizado-bolsonaro-manual.xlsx")
elif perfil == "Geral":
  df = pd.read_excel("outputs/Google/5. Posts+tags-normalizado-geral-manual.xlsx")

In [4]:
posts = pd.read_excel('inputs/Google/2. Posts+tags-normalizado-geral.xlsx')
if(perfil != "Geral"):
  posts= posts.loc[posts["Autor"] == perfil]

In [5]:
#geral = pd.read_excel("outputs/Google/5. Posts+tags-normalizado-geral-manual.xlsx")

In [6]:
df['Cluster Manual - Nome'].unique()

array(['Cluster 0', 'Cluster 7', 'Cluster 1', 'Cluster 2', 'Cluster 3',
       'Cluster 4', 'Cluster 5', 'Cluster 6'], dtype=object)

In [7]:
df['Cluster 2'].unique()

array([0, 7, 5, 3, 4, 8], dtype=int64)

In [8]:
dfs = {}
for cluster in df['Cluster Manual - Nome'].unique():
  df_aux = pd.DataFrame(columns=posts.columns)
  for label in df.loc[df['Cluster Manual - Nome'] == cluster]['Class']:
    df_aux = pd.concat([posts.loc[posts["Class"] == label], df_aux])
  dfs[cluster] = df_aux

C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\1246482612.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_aux = pd.concat([posts.loc[posts["Class"] == label], df_aux])
C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\1246482612.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_aux = pd.concat([posts.loc[posts["Class"] == label], df_aux])
C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\1246482612.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entr

# Verificando se cada cluster está OK

In [9]:
erros_indo = {}
for cluster in df['Cluster Manual - Nome'].unique():
  erros_indo[cluster] = ''
  for label in df.loc[df['Cluster Manual - Nome'] == cluster]['Class'].values:
    if(dfs[cluster].loc[dfs[cluster]['Class'] == label].empty):
      erros_indo[cluster] =f"{label}; {erros_indo[cluster]}"

In [10]:
erros_voltando = {}
for cluster in df['Cluster Manual - Nome'].unique():
  erros_voltando[cluster] = ''
  for label in dfs[cluster]['Class'].unique():
    if(df.loc[(df['Class'] == label) & (df['Cluster Manual - Nome'] == cluster)].empty):
      erros_voltando[cluster] = f"{label}; {erros_voltando[cluster]}"

In [11]:
print("Erros que tiveram indo")
for err in erros_indo:
  print(f"{err}: {erros_indo[err]}")

Erros que tiveram indo
Cluster 0: 
Cluster 7: 
Cluster 1: 
Cluster 2: 
Cluster 3: 
Cluster 4: 
Cluster 5: 
Cluster 6: 


In [12]:
print("Erros que tiveram voltando")
for err in erros_voltando:
  print(f"{err}: {erros_voltando[err]}")

Erros que tiveram voltando
Cluster 0: 
Cluster 7: 
Cluster 1: 
Cluster 2: 
Cluster 3: 
Cluster 4: 
Cluster 5: 
Cluster 6: 


In [13]:
for cluster in df['Cluster Manual - Nome'].unique():
  print(f"{cluster}: {len(dfs[cluster]['ID'].unique())}") 

Cluster 0: 346
Cluster 7: 359
Cluster 1: 682
Cluster 2: 396
Cluster 3: 826
Cluster 4: 625
Cluster 5: 190
Cluster 6: 65


In [14]:
n = 30
dfs_result = {}
for cluster in df['Cluster Manual - Nome'].unique():
  result = pd.DataFrame(columns=posts.columns)
  while len(result['ID'].unique()) < n:
    n_retiradas = n - len(result['ID'].unique())
    result = pd.concat([result, dfs[cluster].sample(n_retiradas).sort_values('ID')])
    result.drop_duplicates('ID', inplace=True)
  dfs_result[cluster] = result

C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\4189407734.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, dfs[cluster].sample(n_retiradas).sort_values('ID')])
C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\4189407734.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, dfs[cluster].sample(n_retiradas).sort_values('ID')])
C:\Users\Admin\AppData\Local\Temp\ipykernel_7028\4189407734.py:7: FutureWarning: The behavior of DataFrame concate

In [15]:
print('Quantidade de IDs únicos em cada cluster')
for cluster in df['Cluster Manual - Nome'].unique():
  print(f"{cluster}: {dfs_result[cluster]['ID'].unique().shape[0]}")

Quantidade de IDs únicos em cada cluster
Cluster 0: 30
Cluster 7: 30
Cluster 1: 30
Cluster 2: 30
Cluster 3: 30
Cluster 4: 30
Cluster 5: 30
Cluster 6: 30


In [16]:
print('Quantidade de linhas em cada cluster')
for cluster in df['Cluster Manual - Nome'].unique():
  print(f"{cluster}: {dfs_result[cluster].shape[0]}")

Quantidade de linhas em cada cluster
Cluster 0: 30
Cluster 7: 30
Cluster 1: 30
Cluster 2: 30
Cluster 3: 30
Cluster 4: 30
Cluster 5: 30
Cluster 6: 30


In [17]:
import shutil
import os

In [18]:
fileId = pd.read_excel('inputs/map-geral-file-id.xlsx')

In [ ]:
if os.path.exists(f"outputs/images/{perfil}"):
  shutil.rmtree(f"outputs/images/{perfil}")
os.mkdir(f"outputs/images/{perfil}")

In [ ]:
for cluster in df['Cluster Manual - Nome'].unique():
  clust = cluster.replace('/', '-')
  if os.path.exists(f"outputs/images/{perfil}/{clust}"):
    shutil.rmtree(f"outputs/images/{perfil}/{clust}")
  os.mkdir(f"outputs/images/{perfil}/{clust}")
  
  for row in range(n):
    if dfs_result[cluster].iloc[row]['Autor'] == 'Lula':
      shutil.copyfile(f"../1. Computer Vision/{fileId.loc[fileId['ID'] == dfs_result[cluster].iloc[row]['ID']]['File'].values[0]}", f"outputs/images/{perfil}/{clust}/{dfs_result[cluster].iloc[row]['ID']}.jpg")
    else:
      shutil.copyfile(f"../1. Computer Vision/{fileId.loc[fileId['ID'] == dfs_result[cluster].iloc[row]['ID']]['File'].values[0]}", f"outputs/images/{perfil}/{clust}/{dfs_result[cluster].iloc[row]['ID']}.jpg")

# Tudo abaixo é codigo antigo

In [ ]:
df = df.groupby(['Cluster Manual - Nome', "Class"]).size().reset_index(name="Total")
df["Cluster Manual - Nome"] = df["Cluster Manual - Nome"].apply(lambda cluster: cluster.replace("/", "-"))

In [ ]:
for cluster in df["Cluster Manual - Nome"].unique():
  file = f"outputs/images/{cluster}"
  test = pd.DataFrame(columns=["ID"] + list(df.loc[df['Cluster Manual - Nome'] == cluster]["Class"].values))
  aux = pd.DataFrame((os.listdir(file)), columns=['ID'])
  aux["ID"] = aux["ID"].apply(lambda id: id.replace(".jpg", ""))
  pd.merge(aux, test, on="ID", how="left").to_excel(file+"/analise.xlsx")

In [ ]:
df.loc[df["Cluster Manual - Nome"] == 'Corpo - Roupa']